In [6]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

In [7]:
import pandas as pd

df = pd.read_csv('flowers/train_labels.csv')

df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,_IGP1402.jpg,4928,3264,flower,3798,1798,4074,2256
1,_IGP1402.jpg,4928,3264,flower,3454,2344,3738,2576
2,_IGP1402.jpg,4928,3264,flower,3662,2266,3918,2494
3,_IGP1402.jpg,4928,3264,flower,3736,2428,3976,2670
4,_IGP1402.jpg,4928,3264,flower,3686,2636,3918,2798


In [3]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

def get_flowers_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['flower']
    df['class'] = df['class'].map(lambda x: 'flower')
    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        
        image = cv2.imread(filename)
        
        height, width, _ = image.shape
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes = ['flower']

for d in ["train", "test"]:
  DatasetCatalog.register('flowers_data/' + d, lambda d=d: get_flowers_dicts('flowers/' + d + '_labels.csv', 'flowers/' + d+'/'))
  MetadataCatalog.get('flowers_data/' + d).set(thing_classes=classes)
flowers_metadata = MetadataCatalog.get('flowers_data/train')


In [5]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
setup_logger()
cfg = get_cfg()
cfg.merge_from_file("./detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "model_final_59f53c.pkl"
cfg.DATASETS.TRAIN = ('flowers_data/train',)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.MAX_ITER = 20000
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.SOLVER.BASE_LR = 0.0001
cfg.MODEL.DEVICE = "cuda:1"
cfg.OUTPUT_DIR = "output_model_flowers"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config ./detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


WARNING [02/02 17:57:31 d2.config.compat]: Config './detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[02/02 17:57:33 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (co

[02/02 17:57:55 d2.data.build]: Removed 0 images with no usable annotations. 79 images left.
[02/02 17:57:55 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   flower   | 737          |
|            |              |
[02/02 17:57:55 d2.data.detection_utils]: TransformGens used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[02/02 17:57:55 d2.data.build]: Using training sampler TrainingSampler


/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/analiza_obrazu/anaconda3/envs/triffid/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

[02/02 17:57:55 d2.engine.train_loop]: Starting training from iteration 0
[02/02 17:58:25 d2.utils.events]: eta: 6:42:28  iter: 19  total_loss: 1.756  loss_cls: 1.172  loss_box_reg: 0.585  time: 1.3751  data_time: 0.6640  lr: 0.000002  max_mem: 0M
[02/02 17:58:54 d2.utils.events]: eta: 7:25:32  iter: 39  total_loss: 1.755  loss_cls: 1.167  loss_box_reg: 0.595  time: 1.4028  data_time: 0.6810  lr: 0.000004  max_mem: 0M
[02/02 17:59:22 d2.utils.events]: eta: 7:40:07  iter: 59  total_loss: 1.709  loss_cls: 1.159  loss_box_reg: 0.547  time: 1.4138  data_time: 0.7029  lr: 0.000006  max_mem: 0M
[02/02 17:59:51 d2.utils.events]: eta: 7:52:20  iter: 79  total_loss: 1.662  loss_cls: 1.141  loss_box_reg: 0.515  time: 1.4173  data_time: 0.7110  lr: 0.000008  max_mem: 0M
[02/02 18:00:20 d2.utils.events]: eta: 7:51:52  iter: 99  total_loss: 1.614  loss_cls: 1.121  loss_box_reg: 0.496  time: 1.4228  data_time: 0.6395  lr: 0.000010  max_mem: 0M
[02/02 18:00:48 d2.utils.events]: eta: 7:51:23  iter: 11

...

[02/03 00:44:19 d2.utils.events]: eta: 0:59:49  iter: 17119  total_loss: 0.045  loss_cls: 0.013  loss_box_reg: 0.032  time: 1.4233  data_time: 0.5431  lr: 0.000100  max_mem: 0M
[02/03 00:44:47 d2.utils.events]: eta: 0:59:24  iter: 17139  total_loss: 0.042  loss_cls: 0.015  loss_box_reg: 0.029  time: 1.4233  data_time: 0.5410  lr: 0.000100  max_mem: 0M
[02/03 00:45:15 d2.utils.events]: eta: 0:58:59  iter: 17159  total_loss: 0.041  loss_cls: 0.013  loss_box_reg: 0.027  time: 1.4232  data_time: 0.5521  lr: 0.000100  max_mem: 0M
[02/03 00:45:43 d2.utils.events]: eta: 0:58:34  iter: 17179  total_loss: 0.040  loss_cls: 0.011  loss_box_reg: 0.030  time: 1.4232  data_time: 0.5540  lr: 0.000100  max_mem: 0M
[02/03 00:46:11 d2.utils.events]: eta: 0:58:09  iter: 17199  total_loss: 0.043  loss_cls: 0.014  loss_box_reg: 0.029  time: 1.4232  data_time: 0.5378  lr: 0.000100  max_mem: 0M
[02/03 00:46:38 d2.utils.events]: eta: 0:57:44  iter: 17219  total_loss: 0.043  loss_cls: 0.014  loss_box_reg: 0.03

[02/03 01:05:19 d2.utils.events]: eta: 0:40:20  iter: 18019  total_loss: 0.045  loss_cls: 0.014  loss_box_reg: 0.030  time: 1.4221  data_time: 0.5629  lr: 0.000100  max_mem: 0M
[02/03 01:05:48 d2.utils.events]: eta: 0:39:56  iter: 18039  total_loss: 0.043  loss_cls: 0.015  loss_box_reg: 0.029  time: 1.4221  data_time: 0.5772  lr: 0.000100  max_mem: 0M
[02/03 01:06:15 d2.utils.events]: eta: 0:39:31  iter: 18059  total_loss: 0.044  loss_cls: 0.013  loss_box_reg: 0.030  time: 1.4220  data_time: 0.5156  lr: 0.000100  max_mem: 0M
[02/03 01:06:43 d2.utils.events]: eta: 0:39:07  iter: 18079  total_loss: 0.041  loss_cls: 0.015  loss_box_reg: 0.027  time: 1.4220  data_time: 0.5526  lr: 0.000100  max_mem: 0M
[02/03 01:07:11 d2.utils.events]: eta: 0:38:42  iter: 18099  total_loss: 0.042  loss_cls: 0.012  loss_box_reg: 0.028  time: 1.4220  data_time: 0.5666  lr: 0.000100  max_mem: 0M
[02/03 01:07:40 d2.utils.events]: eta: 0:38:18  iter: 18119  total_loss: 0.047  loss_cls: 0.015  loss_box_reg: 0.03

[02/03 01:26:20 d2.utils.events]: eta: 0:22:59  iter: 18919  total_loss: 0.039  loss_cls: 0.013  loss_box_reg: 0.027  time: 1.4210  data_time: 0.5339  lr: 0.000100  max_mem: 0M
[02/03 01:26:48 d2.utils.events]: eta: 0:22:34  iter: 18939  total_loss: 0.040  loss_cls: 0.011  loss_box_reg: 0.029  time: 1.4210  data_time: 0.5542  lr: 0.000100  max_mem: 0M
[02/03 01:27:16 d2.utils.events]: eta: 0:22:08  iter: 18959  total_loss: 0.042  loss_cls: 0.013  loss_box_reg: 0.027  time: 1.4210  data_time: 0.5523  lr: 0.000100  max_mem: 0M
[02/03 01:27:44 d2.utils.events]: eta: 0:21:43  iter: 18979  total_loss: 0.042  loss_cls: 0.014  loss_box_reg: 0.029  time: 1.4210  data_time: 0.5676  lr: 0.000100  max_mem: 0M
[02/03 01:28:12 d2.utils.events]: eta: 0:21:02  iter: 18999  total_loss: 0.042  loss_cls: 0.012  loss_box_reg: 0.029  time: 1.4209  data_time: 0.5732  lr: 0.000100  max_mem: 0M
[02/03 01:28:40 d2.utils.events]: eta: 0:20:37  iter: 19019  total_loss: 0.042  loss_cls: 0.013  loss_box_reg: 0.03

[02/03 01:47:25 d2.utils.events]: eta: 0:03:46  iter: 19819  total_loss: 0.044  loss_cls: 0.012  loss_box_reg: 0.026  time: 1.4203  data_time: 0.5604  lr: 0.000100  max_mem: 0M
[02/03 01:47:53 d2.utils.events]: eta: 0:03:21  iter: 19839  total_loss: 0.041  loss_cls: 0.012  loss_box_reg: 0.029  time: 1.4203  data_time: 0.5485  lr: 0.000100  max_mem: 0M
[02/03 01:48:21 d2.utils.events]: eta: 0:02:56  iter: 19859  total_loss: 0.040  loss_cls: 0.011  loss_box_reg: 0.028  time: 1.4203  data_time: 0.5755  lr: 0.000100  max_mem: 0M
[02/03 01:48:49 d2.utils.events]: eta: 0:02:31  iter: 19879  total_loss: 0.038  loss_cls: 0.012  loss_box_reg: 0.025  time: 1.4202  data_time: 0.5307  lr: 0.000100  max_mem: 0M
[02/03 01:49:17 d2.utils.events]: eta: 0:02:06  iter: 19899  total_loss: 0.043  loss_cls: 0.014  loss_box_reg: 0.028  time: 1.4202  data_time: 0.5564  lr: 0.000100  max_mem: 0M
[02/03 01:49:45 d2.utils.events]: eta: 0:01:41  iter: 19919  total_loss: 0.039  loss_cls: 0.011  loss_box_reg: 0.02

OrderedDict()